In [1]:
import queue
import numpy as np
from typing import List, Tuple
import math
import datetime
import pandas as pd
import itertools
from random_gen import RandomNumberGenerator
from gen_algorithm import GeneticKnapsack
from typing import Literal
import time
from multiprocessing import Pool, cpu_count

seed = 42
np.random.seed(seed)
random_gen = RandomNumberGenerator(seedVaule=seed)

In [2]:
n = 1000
C, W, B = random_gen.generate_input(n=n)

population_size = 100
generations = 5000
mutation_rate = 0.1
crossover_rate = 0.7

selection_method: Literal['Tournament', 'Roulette'] = 'Tournament'
crossover_method: Literal['SinglePoint', 'TwoPoint'] = 'SinglePoint'

tournament_size = 5

gen = GeneticKnapsack(costs=C, weights=W, max_weight=B,
                      population_size=population_size,
                      generations=generations,
                      mutation_rate=mutation_rate,
                      crossover_rate=crossover_rate,
                      selection_method=selection_method,
                      tournament_size=tournament_size,
                      crossover_method=crossover_method)

solution_cost, solution_weight, solution_costs, solution_weights = gen.run()
print(f"=======================================")
print(f"Solution cost: {solution_cost}")
print(f"Solution weight: {solution_weight}")

Solution cost: 3710
Solution weight: 2657


In [ ]:
n = 200
C, W, B = random_gen.generate_input(n=n)

population_size = 100
generations = 5000
mutation_rate = [0.05, 0.1, 0.15]
crossover_rate = [0.6, 0.7, 0.8]

selection_method: Literal['Tournament', 'Roulette'] = 'Roulette'
crossover_method: Literal['SinglePoint', 'TwoPoint'] = 'SinglePoint'
tournament_size = 5

reps = 5

res = []

for rep in range(reps):
    for mutation in mutation_rate:
        for crossover in crossover_rate:
            gen = GeneticKnapsack(costs=C, weights=W, max_weight=B,
                                population_size=population_size,
                                generations=generations,
                                mutation_rate=mutation,
                                crossover_rate=crossover,
                                selection_method=selection_method,
                                tournament_size=tournament_size,
                                crossover_method=crossover_method)

            solution_cost, solution_weight, solution_costs, solution_weights = gen.run()
            res.append([rep, mutation, crossover, solution_cost, solution_weight])
            
            print(f"Rep: {rep}, Mutation Rate: {mutation}, Crossover Rate: {crossover}, "
                  f"Solution Cost: {solution_cost}, Solution Weight: {solution_weight}")
            df = pd.DataFrame(res, columns=[
                'Rep', 'MutationRate', 'CrossoverRate', 'SolutionCost', 'Solution Weight'])
            df.to_csv('results/test_params.csv', index=False)


Rep: 0, Mutation Rate: 0.05, Crossover Rate: 0.6, Solution Cost: 689, Solution Weight: 549
Rep: 0, Mutation Rate: 0.05, Crossover Rate: 0.7, Solution Cost: 684, Solution Weight: 552
Rep: 0, Mutation Rate: 0.05, Crossover Rate: 0.8, Solution Cost: 702, Solution Weight: 552
Rep: 0, Mutation Rate: 0.1, Crossover Rate: 0.6, Solution Cost: 701, Solution Weight: 553
Rep: 0, Mutation Rate: 0.1, Crossover Rate: 0.7, Solution Cost: 689, Solution Weight: 549
Rep: 0, Mutation Rate: 0.1, Crossover Rate: 0.8, Solution Cost: 715, Solution Weight: 549
Rep: 0, Mutation Rate: 0.15, Crossover Rate: 0.6, Solution Cost: 687, Solution Weight: 555
Rep: 0, Mutation Rate: 0.15, Crossover Rate: 0.7, Solution Cost: 703, Solution Weight: 555
Rep: 0, Mutation Rate: 0.15, Crossover Rate: 0.8, Solution Cost: 698, Solution Weight: 553
Rep: 1, Mutation Rate: 0.05, Crossover Rate: 0.6, Solution Cost: 708, Solution Weight: 553
Rep: 1, Mutation Rate: 0.05, Crossover Rate: 0.7, Solution Cost: 688, Solution Weight: 555
Re

In [5]:
n_sizes = [50, 100, 150, 200, 250, 300]

population_size = 100
generations = 5000
mutation_rate = 0.1
crossover_rate = 0.8

selection_method: Literal['Tournament', 'Roulette'] = 'Roulette'
tournament_size = 5

reps = 5

crossover_methods = ['SinglePoint', 'TwoPoint']

res = []

for rep in range(reps):
    for n in n_sizes:
        for cross in crossover_methods:
            C, W, B = random_gen.generate_input(n=n)
            gen = GeneticKnapsack(costs=C, weights=W, max_weight=B,
                                population_size=population_size,
                                generations=generations,
                                mutation_rate=mutation_rate,
                                crossover_rate=crossover_rate,
                                selection_method=selection_method,
                                tournament_size=tournament_size,
                                crossover_method=cross)

            solution_cost, solution_weight, solution_costs, solution_weights = gen.run()
            res.append([rep, n, cross, solution_cost, solution_weight])
            
            print(f"Rep: {rep}, N={n}, Crossover Method: {cross}, "f"Solution Cost: {solution_cost}, Solution Weight: {solution_weight}")   
            
            df = pd.DataFrame(res, columns=[
                'Rep', 'N', 'CrossoverMethod', 'SolutionCost', 'Solution Weight'])
            df.to_csv('results/test_crossover.csv', index=False)     
        

Rep: 0, N=50, Crossover Method: SinglePoint, Solution Cost: 237, Solution Weight: 154
Rep: 0, N=50, Crossover Method: TwoPoint, Solution Cost: 206, Solution Weight: 133
Rep: 0, N=100, Crossover Method: SinglePoint, Solution Cost: 339, Solution Weight: 270
Rep: 0, N=100, Crossover Method: TwoPoint, Solution Cost: 393, Solution Weight: 256
Rep: 0, N=150, Crossover Method: SinglePoint, Solution Cost: 520, Solution Weight: 414
Rep: 0, N=150, Crossover Method: TwoPoint, Solution Cost: 551, Solution Weight: 433
Rep: 0, N=200, Crossover Method: SinglePoint, Solution Cost: 692, Solution Weight: 530
Rep: 0, N=200, Crossover Method: TwoPoint, Solution Cost: 734, Solution Weight: 534
Rep: 0, N=250, Crossover Method: SinglePoint, Solution Cost: 870, Solution Weight: 690
Rep: 0, N=250, Crossover Method: TwoPoint, Solution Cost: 858, Solution Weight: 723
Rep: 0, N=300, Crossover Method: SinglePoint, Solution Cost: 1007, Solution Weight: 841
Rep: 0, N=300, Crossover Method: TwoPoint, Solution Cost: 9

In [10]:
n_sizes = [50, 100, 150, 200, 250, 300]

population_size = 100
generations = 5000
mutation_rate = 0.1
crossover_rate = 0.8

crossover_method: Literal['SinglePoint', 'TwoPoint'] = 'SinglePoint'


reps = 5


selections = [
    ('Tournament', None),
    ('Roulette', None)
]

res = []

for rep in range(reps):
    for n in n_sizes:
        for method, t_size in selections:
            C, W, B = random_gen.generate_input(n=n)  
            
            selection_name = f'{method}' if t_size is None else f'{method} x {t_size}'
            
            gen = GeneticKnapsack(costs=C, weights=W, max_weight=B,
                                population_size=population_size,
                                generations=generations,
                                mutation_rate=mutation_rate,
                                crossover_rate=crossover_rate,
                                selection_method=method,
                                tournament_size=t_size,
                                crossover_method=crossover_method)
            solution_cost, solution_weight, solution_costs, solution_weights = gen.run()
            res.append([rep, n, selection_name, solution_cost, solution_weight])
            print(f"Rep: {rep}, N={n}, Selection Method: {selection_name}, "
                  f"Solution Cost: {solution_cost}, Solution Weight: {solution_weight}")
            df = pd.DataFrame(res, columns=[
                'Rep', 'N', 'SelectionMethod', 'SolutionCost', 'Solution Weight'])
            df.to_csv('results/test_selection.csv', index=False)

Rep: 0, N=50, Selection Method: Tournament, Solution Cost: 224, Solution Weight: 123
Rep: 0, N=50, Selection Method: Roulette, Solution Cost: 219, Solution Weight: 144
Rep: 0, N=100, Selection Method: Tournament, Solution Cost: 378, Solution Weight: 259
Rep: 0, N=100, Selection Method: Roulette, Solution Cost: 383, Solution Weight: 267
Rep: 0, N=150, Selection Method: Tournament, Solution Cost: 643, Solution Weight: 391
Rep: 0, N=150, Selection Method: Roulette, Solution Cost: 575, Solution Weight: 410
Rep: 0, N=200, Selection Method: Tournament, Solution Cost: 787, Solution Weight: 520
Rep: 0, N=200, Selection Method: Roulette, Solution Cost: 795, Solution Weight: 554
Rep: 0, N=250, Selection Method: Tournament, Solution Cost: 929, Solution Weight: 695
Rep: 0, N=250, Selection Method: Roulette, Solution Cost: 912, Solution Weight: 646
Rep: 0, N=300, Selection Method: Tournament, Solution Cost: 1114, Solution Weight: 862
Rep: 0, N=300, Selection Method: Roulette, Solution Cost: 1017, S

KeyboardInterrupt: 